<a href="https://colab.research.google.com/github/lauraluebbert/test_gget_alphafold/blob/main/test_gget_alphafold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [gget alphafold](https://github.com/pachterlab/gget) test

___

Install and import gget:

In [ ]:
!pip install -U -q gget
import gget

Use Miniconda to install OpenMM:  

In [ ]:
# For pretty plots
%config InlineBackend.figure_format='retina'

# Install Miniconda
import sys
!rm -rf /opt/conda
!wget -q -P /tmp \
  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh \
    && bash /tmp/Miniconda3-latest-Linux-x86_64.sh -b -p /opt/conda \
    && rm /tmp/Miniconda3-latest-Linux-x86_64.sh

# Install openmm v7.7.0
PATH=%env PATH
%env PATH=/opt/conda/bin:{PATH}
!conda install -qy conda==23.5.2 && conda install -qy -c conda-forge \
      python=3.10 \
      openmm=7.7.0

# Uninstall pyOpenSSL to avoid dependecy conflicts
!pip uninstall -q -y pyOpenSSL

# Add to path
sys.path.append('/opt/conda/lib/python3.10/site-packages')

Install third-part dependencies and download AlphaFold model parameters using `gget setup` (this might take a few minutes):

In [ ]:
gget.setup("alphafold")

Predict the 3D structure of CASP14 target [T1024](https://predictioncenter.org/casp14/target.cgi?id=8&view=all):

In [ ]:
gget.alphafold(
    "MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH",
    relax=True,
    out=None
    )